In [ ]:
from langchain_openai import ChatOpenAI
import os


if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = ""

In [2]:
llm = ChatOpenAI(model="gpt-4o")

# LangChain을 활용한 챗봇 구현

### 1. Retrieval augmented generation

- 문서를 바탕으로 질의 응답을 할 수 있는 봇을 만듭니다.

### 2. 대화 히스토리를 고려한 RAG 챗봇

- 단발성 질문이 아닌, 과거 질문 내역을 바탕으로 질문을 이어갈 수 있게끔 history를 추가합니다.

---

## 1. Retrieval augmented generation

다음과 같은 기능을 가진 챗봇을 만들 예정입니다.

1. 문서를 기반으로 관련성 있는 정보를 가져오는 봇
2. 관련 정보를 바탕으로 사용자의 질문에 답변을 하는 봇

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Maximizing Muscle Hypertrophy.pdf")
pages = loader.load_and_split()

- PDF를 읽어 LangChain의 다른 컴포넌트에서 사용할 수 있는 Document의 형태로 변환합니다.
- 읽는 파일이나 데이터의 종류에 따라 사용해야 하는 loader가 달라집니다.

In [12]:
pages[0]

Document(metadata={'producer': 'iText® 7.1.1 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'PyPDF', 'creationdate': '2019-12-06T03:19:50+01:00', 'moddate': '2019-12-06T03:19:50+01:00', 'source': 'Maximizing Muscle Hypertrophy.pdf', 'total_pages': 14, 'page': 0, 'page_label': '1'}, page_content='International  Journal  of \nEnvironmental Research\nand Public Health\nReview\nMaximizing Muscle Hypertrophy: A Systematic\nReview of Advanced Resistance Training Techniques\nand Methods\nMichal Krzysztoﬁk *\n , Michal Wilk\n , Grzegorz Wojdała\n and Artur Goła´ s\nInstitute of Sport Sciences, Jerzy Kukuczka Academy of Physical Education in Katowice, ul. Mikolowska 72a,\n40-065 Katowice, Poland; m.wilk@awf.katowice.pl (M.W.); wojdala.grzegorz@gmail.com (G.W.);\na.golas@awf.katowice.pl (A.G.)\n* Correspondence: m.krzysztoﬁk@awf.katowice.pl\nReceived: 12 October 2019; Accepted: 3 December 2019; Published: 4 December 2019\n/gid00030/gid00035/gid00032/gid00030/gid00038/gid00001/gid00033/gid

- 텍스트를 LLM에 활용하기 위해서는 사용자의 '질문'과 연관된 정보를 외부에서 제공해줄 수 있어야 합니다.
- '질문'과의 연관성을 측정하기 위한 방법이 embedding 유사도 측정입니다.
- 텍스트를 벡터의 형태로 변환하고, 벡터간의 similarity를 측정함으로써 두 텍스트의 '연관성'을 숫자로 표현할 수 있습니다.
- 따라서 활용하고자 하는 파일에 있는 텍스트를 벡터로 변환하고 저장해야 활용할 수 있습니다.
    - 가장 첫 번째 단계는 텍스트를 split하여 저장하는 것입니다.

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)

In [ ]:
splits[0]

<span style="background-color: #A5B68D; border-radius: 5px; padding: 2px 6px; border: 1px solid #ccc; font-family: sans-serif;">
    retriever</span>에는 아주 많은 기능이 내포되어 있습니다.
코드를 실행함으로서 다음 스텝들이 내부에서 실행됩니다.

1. Document 내에 있는 text를 추출
2. 추가할 meta data가 있다면 추가
3. Text를 embedding vector로 변환 (이때 embedding model 활용)
4. Embedding vector를 vectorDB에 적재

심화된 내용을 공부하고 싶으시면 : Retrieval augmented generation (RAG)

In [5]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
# -- 여러 문서 정보를 input으로 받아서 이를 연결하는 작업을 수행

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain 
# -- vector DB 에서 데이터 검색 및 llm통해 답변 생성까지 할 수 있도록 체인

ModuleNotFoundError: No module named 'langchain_chroma'

In [ ]:
## 00 청크로 쪼갠 문서 정보 -> vector 로 변환해서 저장
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
  ## from_documents : txt > emb vector
  ## documents : 입력하고자 하는 문서 정보 (청크 단위로 쪼개진 문자열 리스트)
  ## embedding: embedding 모델의 정의

retriever = vectorstore.as_retriever()
  ## 크로마 벡터 디비에 저장된 문서 벡터 정보를 가지고 유사한 문서를 검색하는 객체 (검색기) 생성


`create_stuff_documents_chain`는 이전에 텍스트에서 변환된 document 객체를 chain에 연결하기 위한 wrapper 입니다.

따라서 llm, prompt와 함께 retriever도 chain에 연결하여 실행 할 수 있도록 합니다.

`create_retrieval_chain`는 이렇게 준비된 chain들을 모두 연결합니다.

In [ ]:
system_prompt = (
    """당신은 질문-답변을 담당하는 전문가 입니다. 다음 정보를 활용하여 질문에 답을 하시오. 
    모르면 모른다고 답하고, 답변은 간결하게 하시오.
    {context}"""
)

# 템플릿 객체 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

## retriever 의 결과 (가장 유사도 높은 정보들) 를 바탕으로 프롬프트를 완성시키고 llm 에 전달함. 
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
response = rag_chain.invoke({"input": "현재 논문의 주제가 뭐야?"})
response["answer"]

# retriever | prompt | llm 

현재는 `retriever` -> `prompt` -> `llm` 순서대로 정보가 흐릅니다.

retriever를 통해 정보가 prompt에 전달이 되고, 완성된 prompt가 LLM에게 전달이 되어 답변이 생성됩니다.

## 2. 대화 히스토리를 고려한 RAG 챗봇

In [ ]:
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)

In [ ]:
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "논문 리뷰 전문가 입니다. 사용자의 질문에 답하세요. {context}"),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
store = {}

# session_id를 key로 삼아 dictionary에 저장
# value 값에는 chat history를 저장
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
conversational_rag_chain.invoke(
    {"input": "논문의 주제가 뭐야?"},
    config={
        "configurable": {"session_id": "paperbot"}
    },
)["answer"]

In [ ]:
conversational_rag_chain.invoke(
    {"input": "논문에서 이야기 하는 근육을 키우는 가장 좋은 방법은 뭐야?"},
    config={
        "configurable": {"session_id": "paperbot"}
    },
)["answer"]

In [ ]:
store


---
<span style="color:rgb(120, 120, 120)">본 학습 자료를 포함한 사이트 내 모든 자료의 저작권은 엘리스에 있으며 외부로의 무단 복제, 배포 및 전송을 불허합니다.

Copyright @ elice all rights reserved</span>